In [ ]:
import json
import os

from openai import OpenAI
from pydantic import BaseModel, Field

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Let's define the knowledge base retrieval tool

In [ ]:
def search_kb(question: str):
    """
    Load the whole knowledge base from the JSON file.
    (This is a mock function for demonstration purposes, we don't search)
    """
    with open("knowledge_base.json", "r") as f:
        return json.load(f)

In [ ]:
# Step 1: Now, we call the model with search_kb tool that we defined
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_kb",
            "description": "Get the answer to the user's question from the knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"},
                },
                "required": ["question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful assistant that answers questions from the knowledge base about our e-commerce store."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the return policy?"},
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [ ]:
#Step 2: Model decides to call the function
completion.model_dump()

In [ ]:
#Step 3: Now we execute search_kb function
def call_function(name, args):
    if name == "search_kb":
        return search_kb(**args)


for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )

In [ ]:
#Step 4: Pass on the result and call the model again

class KBResponse(BaseModel):
    answer: str = Field(description="The answer to the user's question.")
    source: int = Field(description="The record id of the answer.")


completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    response_format=KBResponse,
)

In [ ]:
# Step 5: Check model response

final_response = completion_2.choices[0].message.parsed
final_response.answer
final_response.source

In [ ]:
#Questions that won't trigger the tool

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the weather in Tokyo?"},
]

completion_3 = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

completion_3.choices[0].message.content